# Forecasting
Contents:
- Univariate time series forecasting
- Univariate time series forecasting with exogenous variables
- Multivariate time series forecasting


In [1]:
from warnings import simplefilter
simplefilter(action="ignore", category=RuntimeWarning)

## Univariate forecasting
In forecasting, we are interested in using past data to make temporal forward predictions. Sktime provides common statistical forecasting algorithms and tools for building composite machine learning models.

The **basic workflow** for forecasting is generating one single prediction for a single point in time
1. Specify the data
2. Specify the task
3. Split the data
4. Specify the model
5. Fit
6. Predict
7. Evaluate

### 1. Specify the data


In [2]:
from sktime.datasets import load_shampoo_sales
from sktime.utils.plotting import plot_series

y = load_shampoo_sales()

plot_series(y)

(<Figure size 4200x1050 with 1 Axes>,
 <AxesSubplot:ylabel='Number of shampoo sales'>)

<Figure size 4200x1050 with 1 Axes>

### 2. Task specification
Define a forecasting task; in the example:
- Try to predict the last 6 months of data, using the previous data as training data. Each point in the time series represents a month, so the last 6 points should be hold out as test data, and a 6-step ahead forecasting horizon should be used to evaluate the forecasting performance.
- The metric chosen to quantify the accuracy of the forecast is MAPE (mean absolute percentage error). A lower MAPE means higher accuracy.

#### Forecasting horizon
When forecasts are generated, a forecasting horizon has to be specified and passed as a parameter to the forecasting algorithm. The forecasting horizon can be specified as a numpy array of the steps ahead relative to the end of the training series:

**Using an array of values**


In [3]:
import numpy as np

fh = np.arange(6) + 1
fh

array([1, 2, 3, 4, 5, 6])

**Using the ForecastingHorizon object**


In [4]:
import pandas as pd
from sktime.forecasting.base import ForecastingHorizon

fh = ForecastingHorizon(
    pd.period_range('1993-07', periods=6, freq='M'),
    is_relative=False
)
fh

ForecastingHorizon(['1993-07', '1993-08', '1993-09', '1993-10', '1993-11', '1993-12'], dtype='period[M]', is_relative=False)

In [5]:
cutoff = pd.Period('1993-06', freq='M')
fh.to_relative(cutoff)

ForecastingHorizon([1, 2, 3, 4, 5, 6], dtype='int64', is_relative=True)

### 3. Split the data
Split the data into train, test, and split datasets


In [6]:
from sktime.forecasting.model_selection import temporal_train_test_split

y_train, y_test = temporal_train_test_split(y, fh=fh)

plot_series(y_train, y_test, labels=['y_train', 'y_test'])

(<Figure size 4200x1050 with 1 Axes>,
 <AxesSubplot:ylabel='Number of shampoo sales'>)

<Figure size 4200x1050 with 1 Axes>

### 4. Model specification


In [7]:
from sktime.forecasting.naive import NaiveForecaster

forecaster = NaiveForecaster(strategy='drift', window_length=10)

### 5. Fitting


In [8]:
forecaster.fit(y_train)

NaiveForecaster(strategy='drift', window_length=10)

Notice that the `fit` method is different from the one in sklearn


In [9]:
forecaster.fit?

1. the first argument is the target series `y`
2. the second argument is exogenous data (see below)
3. the third argument is the forecasting horizon; it is not required for all models to pass the forecasting horizon in the fitting step 

### 6. Predicting


In [10]:
y_pred = forecaster.predict(fh)

### 7. Evaluating


In [11]:
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

mean_absolute_percentage_error(y_test, y_pred, symmetric=False)

0.16469764622516225

In [12]:
plot_series(y_train, y_test, y_pred, labels=['y_train','y_test','y_pred'])

(<Figure size 4200x1050 with 1 Axes>,
 <AxesSubplot:ylabel='Number of shampoo sales'>)

<Figure size 4200x1050 with 1 Axes>

### Another example using the ARIMA model


In [13]:
from sktime.datasets import load_shampoo_sales
from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.arima import AutoARIMA
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
import numpy as np

# data load
y = load_shampoo_sales()

# task specification: forecasting 6 months ahead
fh = np.arange(6) + 1

# split the data
y_train, y_test = temporal_train_test_split(y, fh=fh)

# model specification
forecaster = AutoARIMA(sp=12, suppress_warnings=True)

# model fit
forecaster.fit(y_train)

# model predict
y_pred = forecaster.predict(fh)

# model evaluation
plot_series(y_train, y_test, y_pred, labels=['y_train','y_test','y_pred'])

(<Figure size 4200x1050 with 1 Axes>,
 <AxesSubplot:ylabel='Number of shampoo sales'>)

<Figure size 4200x1050 with 1 Axes>

In [14]:
mean_absolute_percentage_error(y_test, y_pred, symmetric=False)

0.23140725522776395

### Summary of the basic workflow
- single, fixed cutoff point at which predictions are generated
- common interface for forecasters

## Forecasters in sktime
The complete list of sktime estimators is available [here](https://www.sktime.org/en/stable/estimator_overview.html)


In [15]:
from sktime.registry import all_estimators

# all_estimators('forecaster', as_dataframe=True)

### Is it possible to just use scikit-learn?
Using sklearn alone with time series data tasks has many pitfalls. It is better to use it together with sktime, since it provides a meta-estimator for this approach, that is:
- **modular** and **compatible with scikit-learn**, so that it is possible to easily apply any scikt-learn regressor to solve a forecasting problem
- **parametric** and **tuneable**, allowing to tune hyperparameters like the window length or the strategy to generate forecasts
- **adaptive**, in the sense that it adapts the scikit-learn estimator interface to that of a forecaster, making sure that the model can be properly tuned and evaluated


In [16]:
from sktime.datasets import load_airline
from sktime.utils.plotting import plot_series

# data load
y = load_airline()
plot_series(y)

(<Figure size 4200x1050 with 1 Axes>,
 <AxesSubplot:ylabel='Number of airline passengers'>)

<Figure size 4200x1050 with 1 Axes>

In [17]:
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sklearn.neighbors import KNeighborsRegressor
from sktime.forecasting.compose import make_reduction

# model task and data split
y_train, y_test = temporal_train_test_split(y, test_size=12)
fh = ForecastingHorizon(y_test.index, is_relative=False)

# model specification
regressor = KNeighborsRegressor(n_neighbors=3)
forecaster = make_reduction(regressor, strategy='recursive', window_length=12) 

# model fit
forecaster.fit(y_train, fh=fh)

# model predict
y_pred = forecaster.predict()

# model evaluation
plot_series(y_train, y_test, y_pred, labels=['y_train','y_test','y_pred'])

(<Figure size 4200x1050 with 1 Axes>,
 <AxesSubplot:ylabel='Number of airline passengers'>)

<Figure size 4200x1050 with 1 Axes>

In [18]:
mean_absolute_percentage_error(y_test, y_pred, symmetric=False)

0.12825559519541507

Notice that the function `make_reduction` wraps around a regressor and makes it possible to use it as a forecaster
